In [1]:
import os
import datetime

from vllm.steer_vectors.request import SteerVectorRequest
from vllm import LLM, SamplingParams

from easysteer.steer import StatisticalControlVector

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-15 15:27:38 [__init__.py:244] Automatically detected platform cuda.


In [2]:
model_path = "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
# vector_path="/home/meixinyu/EasySteer/vectors/thinking_switch_pca_MATH-500.gguf" #change to your path
vector_path="MATH500.gguf" #change to your path

control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='diffmean', directions={0: memmap([-0.37762213,  0.3621289 ,  0.07671386, ...,  0.09187746,
         0.01702148, -0.02408692], shape=(1536,), dtype=float32), 1: memmap([-0.37708008,  0.3185938 ,  0.00083008, ...,  0.10798828,
        -0.12316895,  0.07854492], shape=(1536,), dtype=float32), 2: memmap([-0.50412107,  0.2696436 , -0.17655273, ...,  0.13890624,
        -0.12346679,  0.2603125 ], shape=(1536,), dtype=float32), 3: memmap([-0.6581836 ,  0.06841797, -0.3773291 , ...,  0.09156249,
        -0.11788085,  0.07102537], shape=(1536,), dtype=float32), 4: memmap([-0.67054677, -0.11105469, -0.1010547 , ..., -0.02114257,
        -0.01477537, -0.14418948], shape=(1536,), dtype=float32), 5: memmap([-0.7164061 , -0.07879883, -0.20362793, ...,  0.02737793,
        -0.156875  ,  0.0581836 ], shape=(1536,), dtype=float32), 6: memmap([-0.2943945 , -0.01407228, -0.28964844, ...,  0.01804686,
         0.31013674, -0.04800782], shape=(1536,), 

In [3]:
steer_vector_request_pos = SteerVectorRequest(
            steer_vector_name="fast",
            steer_vector_id=1,
            steer_vector_local_path=vector_path,
            scale=1.0,
            target_layers=[21],
            prefill_trigger_tokens=[-1],
            prefill_trigger_positions=[-1],
            generate_trigger_tokens=[-1],
            debug=False,
            algorithm='direct'
        )
steer_vector_request_neg = SteerVectorRequest(
            steer_vector_name="slow",
            steer_vector_id=2,
            steer_vector_local_path=vector_path,
            scale=-1.0,
            target_layers=[21],
            prefill_trigger_tokens=[-1],
            prefill_trigger_positions=[-1],
            generate_trigger_tokens=[-1],
            debug=False,
            algorithm='direct'
        )

In [4]:
sampling_params = SamplingParams(temperature=0.0,max_tokens=4096)
prompt_template = "<|User|>%s<|Assistant|><think>\n"
prompt = "Find the constant term in the expansion of $$\\left(10x^3-\\frac{1}{2x^2}\\right)^{5}$$"
# prompt = "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."
input = prompt_template % prompt

In [5]:
llm = LLM(model=model_path, enable_steer_vector=True, tensor_parallel_size=1,enforce_eager=True)
output_base = llm.generate(
    input,
    sampling_params
)
generated_text_base = output_base[0].outputs[0].text

output_pos = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_pos
            )
generated_text_pos = output_pos[0].outputs[0].text

# output_neg = llm.generate(
#                 input,
#                 sampling_params,
#                 steer_vector_request=steer_vector_request_neg
#             )
# generated_text_neg = output_neg[0].outputs[0].text

INFO 07-15 15:27:47 [config.py:841] This model supports multiple tasks: {'classify', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 07-15 15:27:47 [config.py:1472] Using max model len 131072
WARNING 07-15 15:27:47 [arg_utils.py:1770] --enable-steer-vector is not supported by the V1 Engine. Falling back to V0. 
WARNING 07-15 15:27:47 [arg_utils.py:1577] The model has a long context length (131072). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.


2025-07-15 15:27:48,072	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 07-15 15:27:48 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-15 15:27:48 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, r

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.01it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.00it/s]


INFO 07-15 15:27:51 [default_loader.py:272] Loading weights took 1.10 seconds


INFO 07-15 15:27:52 [model_runner.py:1255] Model loading took 3.3461 GiB and 1.257748 seconds
INFO 07-15 15:27:56 [worker.py:295] Memory profiling takes 4.00 seconds
INFO 07-15 15:27:56 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-15 15:27:56 [worker.py:295] model weights take 3.35GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 8.10GiB; the rest of the memory reserved for KV Cache is 31.17GiB.
INFO 07-15 15:27:56 [executor_base.py:115] # cuda blocks: 72954, # CPU blocks: 9362
INFO 07-15 15:27:56 [executor_base.py:120] Maximum concurrency for 131072 tokens per request: 8.91x
INFO 07-15 15:28:01 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 9.15 seconds


Processed prompts: 100%|██████████| 1/1 [00:40<00:00, 40.12s/it, est. speed input: 1.15 toks/s, output: 39.63 toks/s]


In [6]:
print(generated_text_base)

Okay, so I need to find the constant term in the expansion of \(\left(10x^3 - \frac{1}{2x^2}\right)^5\). Hmm, let me think about how to approach this. I remember that when expanding a binomial like \((a + b)^n\), each term in the expansion is given by the binomial theorem: \(\binom{n}{k} a^{n - k} b^k\). So, in this case, \(a = 10x^3\) and \(b = -\frac{1}{2x^2}\), and \(n = 5\).

Alright, so each term in the expansion will look like \(\binom{5}{k} (10x^3)^{5 - k} \left(-\frac{1}{2x^2}\right)^k\). I need to find the term where the exponent of \(x\) is zero because that's the constant term. So, let me figure out how to find the value of \(k\) that makes the exponent of \(x\) zero.

First, let's write out the general term:

\[
\binom{5}{k} \cdot (10x^3)^{5 - k} \cdot \left(-\frac{1}{2x^2}\right)^k
\]

Let me simplify this term step by step. Let's handle each part separately.

Starting with \((10x^3)^{5 - k}\):

\[
10^{5 - k} \cdot x^{3(5 - k)} = 10^{5 - k} \cdot x^{15 - 3k}
\]

Next, \(\l

In [7]:
print(generated_text_pos)

Okay, so I need to find the constant term in the expansion of (10x³ - 1/(2x²))⁵. Hmm, let me think about how to approach this. I remember that when expanding a binomial like (a + b)^n, the general term is given by the combination formula: C(n, k) * a^(n - k) * b^k. So, in this case, a is 10x³ and b is -1/(2x²), and n is 5. 

First, I should write down the general term for this expansion. So, the term would be C(5, k) * (10x³)^(5 - k) * (-1/(2x²))^k. I need to find the term where the exponent of x is zero because that's the constant term. So, I need to find the value of k that makes the exponent of x in this term zero.

Let me compute the exponent of x in each part. For (10x³)^(5 - k), the exponent of x is 3*(5 - k). For (-1/(2x²))^k, the exponent of x is -2*k. So, the total exponent of x in the term is 3*(5 - k) - 2*k. I need this to be zero. So, let me set up the equation:

3*(5 - k) - 2*k = 0.

Let me solve this equation step by step. First, expand 3*(5 - k): that's 15 - 3k. Then sub

In [8]:
# print(generated_text_neg)

In [9]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename=f'fractreason_GSM8K_{timestamp}.txt'

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
)
# filename=f'../../temp/fractreason_GSM8K_{timestamp}.txt'
with open(filename, 'w', encoding='utf-8') as f:
    f.write("=== Base ===\n")
    f.write(generated_text_base + "\n\n")
    f.write("=== Positive: Fast thinking ===\n")
    f.write(generated_text_pos + "\n\n")
    # f.write("=== Negative: Slow thinking ===\n")
    # f.write(generated_text_neg + "\n")
    print("asc tokens: ", len(tokenizer.tokenize(output_base[0].outputs[0].text, add_special_tokens=True)))
    print("asc tokens: ", len(tokenizer.tokenize(output_pos[0].outputs[0].text, add_special_tokens=True)))

asc tokens:  2719
asc tokens:  1590
